In [51]:
#转换地名为拼音格式
import pypinyin

list0 =["汉中","辽阳","四平","内江","六盘水","安顺","新余","牡丹江","晋城","自贡","三门峡","赤峰","本溪","防城港","铁岭","随州","广安","广元","天水","遂宁","萍乡","西双版纳","绥化","鹤壁","湘西","松原","阜新","酒泉","张家界","黔西南","保山","昭通","河池","来宾","玉溪","梧州","鹰潭","钦州","云浮","佳木斯","克拉玛依","呼伦贝尔","贺州","通化","阳泉","朝阳","百色","毕节","贵港","丽江","安康","通辽","德宏","朔州","伊犁","文山","楚雄","嘉峪关","凉山","资阳","雅安","普洱","崇左","庆阳","白山","昌吉","白城","定西","喀什","白银","陇南","巴彦淖尔","巴中","鸡西","海东","张掖","商洛","黑河","哈密","吴忠","攀枝花","双鸭山","阿克苏","石嘴山","阿拉善盟","海西","平凉","林芝","固原","武威","儋州","吐鲁番","甘孜","辽源","临夏","铜川","金昌","鹤岗","伊春","中卫","怒江","和田","迪庆","甘南","阿坝","大兴安岭","七台河","山南","日喀则","塔城","博尔塔拉","昌都","阿勒泰","玉树","黄南","果洛","阿里","海北","那曲"]
# ["银川","西宁","海口","洛阳","南通","常州","徐州","潍坊","淄博","绍兴","温州","台州","大庆","鞍山","中山","珠海","汕头","吉林","柳州","拉萨","保定","邯郸","秦皇岛","沧州","鄂尔多斯","东营","威海","济宁","临沂","德州","滨州","泰安","湖州","嘉兴","金华","泰州","镇江","盐城","扬州","桂林","惠州","湛江","江门","茂名","株洲","岳阳","衡阳","宝鸡","宜昌","襄樊","开封","许昌","平顶山","赣州","九江","芜湖","绵阳","齐齐哈尔","牡丹江","抚顺","本溪","丹东","辽阳","锦州","营口","承德","廊坊","邢台","大同","榆林","延安","天水","克拉玛依","喀什","石河子","南阳","濮阳","安阳","焦作","新乡","日照","聊城","枣庄","蚌埠","淮南","马鞍山","连云港","淮安","丽水","衢州","荆州","安庆","景德镇","新余","湘潭","常德","郴州","漳州","清远","揭阳","梅州","肇庆","玉林","北海","德阳","宜宾","遵义","大理"]
list_pinyin = []
for i in list0:
    s = ''
    for j in pypinyin.pinyin(i, style=pypinyin.NORMAL):
        s += ''.join(j)
    list_pinyin.append(s)

In [52]:
import requests,re,os
from lxml import etree
from fontTools.ttLib import TTFont
import xlwt
import webbrowser
import os
from tqdm import tqdm
import time
f = xlwt.Workbook()
# 创建一个sheet
sheet1 = f.add_sheet('new',cell_overwrite_ok=True)
col1 = sheet1.col(0)
col1.width = 256*38
#初始化第一行
row0 = [u'商铺名称',u'类型',u'评分',u'评论',u'人均消费（单位：￥）',u'店铺地址',u'当地地址',u'效果',u'环境',u'服务',u'地点']
for i in range(0,len(row0)):
    sheet1.write(0,i,row0[i])#初始化第一行~

def download_woff(css_url):
    # 获取woff的url
    woff_url = re.findall(r',url\("(//s3plus.*?\.woff)"\);\}', requests.get(css_url).text, re.S)
    # 下载woff文件到本地
    for url in woff_url:
        content = requests.get('http:' + url).content
        filename = url.split('/')[-1]
        with open('./css破解/' + filename, 'wb') as fp:
            fp.write(content)
    return [name.split('/')[-1] for name in woff_url]

character = list(
    '1234567890店中美家馆小车大市公酒行国品发电金心业商司超生装园场食有新限天面工'
    '服海华水房饰城乐汽香部利子老艺花专东肉菜学福饭人百餐茶务通味所山区门药银农龙停尚安'
    '广鑫一容动南具源兴鲜记时机烤文康信果阳理锅宝达地儿衣特产西批坊州牛佳化五米修爱北养'
    '卖建材三会鸡室红站德王光名丽油院堂烧江社合星货型村自科快便日民营和活童明器烟育宾精'
    '屋经居庄石顺林尔县手厅销用好客火雅盛体旅之鞋辣作粉包楼校鱼平彩上吧保永万物教吃设医'
    '正造丰健点汤网庆技斯洗料配汇木缘加麻联卫川泰色世方寓风幼羊烫来高厂兰阿贝皮全女拉成'
    '云维贸道术运都口博河瑞宏京际路祥青镇厨培力惠连马鸿钢训影甲助窗布富牌头四多妆吉苑沙'
    '恒隆春干饼氏里二管诚制售嘉长轩杂副清计黄讯太鸭号街交与叉附近层旁对巷栋环省桥湖段乡'
    '厦府铺内侧元购前幢滨处向座下県凤港开关景泉塘放昌线湾政步宁解白田町溪十八古双胜本'
    '单同九迎第台玉锦底后七斜期武岭松角纪朝峰六振珠局岗洲横边济井办汉代临弄团外塔杨铁浦'
    '字年岛陵原梅进荣友虹央桂沿事津凯莲丁秀柳集紫旗张谷的是不了很还个也这我就在以可到错'
    '没去过感次要比觉看得说常真们但最喜哈么别位能较境非为欢然他挺着价那意种想出员两推做'
    '排实分间甜度起满给热完格荐喝等其再几只现朋候样直而买于般豆量选奶打每评少算又因情找'
    '些份置适什蛋师气你姐棒试总定啊足级整带虾如态且尝主话强当更板知己无酸让入啦式笑赞'
    '片酱差像提队走嫩才刚午接重串回晚微周值费性桌拍跟块调糕')
length = 0
for x in tqdm(range(0,len(list_pinyin))):
    time.sleep(0.001)
    url = 'http://www.dianping.com/'+ str(list_pinyin[x]) +'/ch10/g110o3' # url = 'http://www.dianping.com/changsha/ch10/g110'
    headers={
    'cookie':"_lxsdk_cuid=1789acb1ea1c8-0799a204fdb9ea-5c3f1e49-10ed98-1789acb1ea1c8; _lxsdk=1789acb1ea1c8-0799a204fdb9ea-5c3f1e49-10ed98-1789acb1ea1c8; _hc.v=ebb31d04-778f-02a6-9984-060d56d71344.1617504706; s_ViewType=10; ctu=3a87482384aa8445669918ae73680ee9a42215c28654ba91ce2d4ec9bf99ab35; ll=7fd06e815b796be3df069dec7836c3df; aburl=1; cityid=3; default_ab=index%3AA%3A3%7CshopList%3AA%3A5; switchcityflashtoast=1; cy=17; cye=xian; fspop=test; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1618070722,1618320955,1618320969,1618322864; ua=dpuser_0539703749; uamo=17390601491; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1618331052; _lxsdk_s=178cc01486f-3c-2b2-b9d%7C%7C28",
    'User-Agent':'Opera/9.80 (Android 2.3.4; Linux; Opera mobi/adr-1107051709; U; zh-cn) Presto/2.8.149 Version/11.10',
    'Referer':'http://www.dianping.com/',
    }
    #破解css
    
#     os.chdir("D://360downloads//美团比赛//css破解//") #设置需要查看的目录D:\360downloads\美团比赛\css破解
#     a = [x for x in os.listdir()]
#     for jack in a:
#         os.remove("D://360downloads//美团比赛//css破解//"+jack)

    rep=requests.get(url,headers=headers).text
    css_url='http:'+re.search(r'(//s3plus.*?\.css)',rep).group(1)
    css_dir='./css破解/'
    if not os.path.exists(css_dir):
        os.mkdir(css_dir)
    filenames=os.listdir(css_dir)
    if not filenames:
        #下载woff文件
        filenames=download_woff(css_url)
    data={}

    for filename in filenames:
        font=TTFont(css_dir+filename)
        keys=font.getGlyphOrder()[2:]
        for index,value in enumerate(keys):
            if value[3:] not in data:
                data[value[3:]]=character[index]
    # 5、利用data字典，进行内容替换
    for k, v in data.items():
        rep = rep.replace(f'&#x{k};', v)
    # tree=etree.HTML(rep)
    # li_list=tree.xpath('//div[@class="content"]/div/ul/li')
    # for li in li_list:
    #     title=li.xpath('.//div[@class="tit"]/a/h4/text()')[0]
    #     tag=li.xpath('string(.//div[@class="tag-addr"]/a/span)')
    #     print(title,tag)
    tree=etree.HTML(rep)
    li_list=tree.xpath('//div[@class="content"]/div/ul/li')
    count = 0
    for li in li_list:
        title = li.xpath('.//div[@class="tit"]/a/h4/text()')[0]
        tag = li.xpath('string(.//div[@class="tag-addr"]/a/span)')
        score = li.xpath(
            'string(.//div[@class="star_score score_50  star_score_sml"]/text()|.//div[@class="star_score score_40  star_score_sml"]/text()|.//div[@class="star_score score_45  star_score_sml"]/text())')
        comments = li.xpath('string(.//div[@class="comment"]/a[@class="review-num"]/b)')
        prize = li.xpath('string(.//div[@class="comment"]/a[@class="mean-price"]/b)')
        address = li.xpath('string(.//div[@class="tag-addr"]/a[@data-click-name="shop_tag_region_click"]/span)')
        localaddress = li.xpath('string(.//div[@class="tag-addr"]/span)')
        flavor = li.xpath('string(.//div[@class="txt"]/span[@class="comment-list"]/span[position()=1])')
        environment = li.xpath('string(.//div[@class="txt"]/span[@class="comment-list"]/span[position()=2])')
        service = li.xpath('string(.//div[@class="txt"]/span[@class="comment-list"]/span[position()=3])')
        import re
        flavor = re.sub("口味", '',flavor)
        environment = re.sub("环境", '',environment)
        service = re.sub("服务", '',service)
        list1 = [comments, prize, flavor, environment, service]
        i = 0
        for i in range(0,len(list1)):
            k = list1[i]
            if "货" in k:
                k = re.sub("货", '8',k)
            if "旅" in k:
                k = re.sub("旅", '6',k)
            if "联" in k:
                k = re.sub("联", '9',k)
            list1[i] = k
            i = i + 1
        comments, prize, flavor, environment, service = list1[0],list1[1],list1[2],list1[3],list1[4]
#         print('\t',title,'\t',tag,'\t', score,'\t', comments,'\t', prize,'\t', address,'\t', localaddress,'\t', flavor,'\t', environment,'\t', service)
#         print("~~~"*10)
        prize = prize.replace('￥','')
        a = [title, tag, score, comments, prize, address, localaddress, flavor, environment, service,list0[x]]
        for j in range(0,len(a)):
            sheet1.write(length + 1+count,j,a[j])
        count = count + 1
    length = length + count
f.save("大众点评收集数据.xls")
webbrowser.open('D:/360downloads/美团比赛/大众点评收集数据.xls')









  0%|                                                                                          | 0/119 [00:00<?, ?it/s]







  1%|▋                                                                                 | 1/119 [00:00<01:06,  1.77it/s]







  2%|█▍                                                                                | 2/119 [00:01<01:04,  1.81it/s]







  3%|██                                                                                | 3/119 [00:01<01:01,  1.90it/s]







  3%|██▊                                                                               | 4/119 [00:02<01:00,  1.89it/s]







  4%|███▍                                                                              | 5/119 [00:02<00:59,  1.93it/s]







  5%|████▏                                                                             | 6/119 [00:03<00:57,  1.97it/s]







  6%|████▊                                                                             | 7/119 [00:03<0

 45%|████████████████████████████████████▊                                            | 54/119 [00:27<00:33,  1.96it/s]







 46%|█████████████████████████████████████▍                                           | 55/119 [00:27<00:32,  1.97it/s]







 47%|██████████████████████████████████████                                           | 56/119 [00:28<00:31,  2.00it/s]







 48%|██████████████████████████████████████▊                                          | 57/119 [00:28<00:30,  2.05it/s]







 49%|███████████████████████████████████████▍                                         | 58/119 [00:29<00:29,  2.06it/s]







 50%|████████████████████████████████████████▏                                        | 59/119 [00:29<00:29,  2.04it/s]







 50%|████████████████████████████████████████▊                                        | 60/119 [00:30<00:29,  2.03it/s]







 51%|█████████████████████████████████████████▌                                       | 61/119 [00:30<00:28,  2

 91%|████████████████████████████████████████████████████████████████████████▌       | 108/119 [00:53<00:05,  2.10it/s]







 92%|█████████████████████████████████████████████████████████████████████████▎      | 109/119 [00:54<00:04,  2.05it/s]







 92%|█████████████████████████████████████████████████████████████████████████▉      | 110/119 [00:54<00:04,  2.04it/s]







 93%|██████████████████████████████████████████████████████████████████████████▌     | 111/119 [00:55<00:03,  2.01it/s]







 94%|███████████████████████████████████████████████████████████████████████████▎    | 112/119 [00:56<00:03,  1.86it/s]







 95%|███████████████████████████████████████████████████████████████████████████▉    | 113/119 [00:56<00:03,  1.69it/s]







 96%|████████████████████████████████████████████████████████████████████████████▋   | 114/119 [00:57<00:02,  1.75it/s]







 97%|█████████████████████████████████████████████████████████████████████████████▎  | 115/119 [00:57<00:02,  1

True

In [ ]:
list_pinyin[1]

In [28]:
msn = """汉中、辽阳、四平、内江、六盘水、安顺、新余、牡丹江、晋城、自贡、三门峡、赤峰、本溪、防城港、铁岭、随州、广安、广元、天水、遂宁、萍乡、西双版纳、绥化、鹤壁、湘西、松原、阜新、酒泉、张家界、黔西南、保山、昭通、河池、来宾、玉溪、梧州、鹰潭、钦州、云浮、佳木斯、克拉玛依、呼伦贝尔、贺州、通化、阳泉、朝阳、百色、毕节、贵港、丽江、安康、通辽、德宏、朔州、伊犁、文山、楚雄、嘉峪关、凉山、资阳、雅安、普洱、崇左、庆阳、乌兰察布、白山、昌吉、白城、兴安盟、定西、喀什、白银、陇南、巴彦淖尔、巴中、鸡西、乌海、临沧、海东、张掖、商洛、黑河、哈密、吴忠、攀枝花、双鸭山、阿克苏、石嘴山、阿拉善盟、海西、平凉、林芝、固原、武威、儋州、吐鲁番、甘孜、辽源、临夏、铜川、金昌、鹤岗、伊春、中卫、怒江、和田、迪庆、甘南、阿坝、大兴安岭、七台河、山南、日喀则、塔城、博尔塔拉、昌都、阿勒泰、玉树、海南、黄南、果洛、克孜勒苏、阿里、海北、那曲、三沙"""

In [29]:
import re
msn.replace('、','","')

'汉中","辽阳","四平","内江","六盘水","安顺","新余","牡丹江","晋城","自贡","三门峡","赤峰","本溪","防城港","铁岭","随州","广安","广元","天水","遂宁","萍乡","西双版纳","绥化","鹤壁","湘西","松原","阜新","酒泉","张家界","黔西南","保山","昭通","河池","来宾","玉溪","梧州","鹰潭","钦州","云浮","佳木斯","克拉玛依","呼伦贝尔","贺州","通化","阳泉","朝阳","百色","毕节","贵港","丽江","安康","通辽","德宏","朔州","伊犁","文山","楚雄","嘉峪关","凉山","资阳","雅安","普洱","崇左","庆阳","乌兰察布","白山","昌吉","白城","兴安盟","定西","喀什","白银","陇南","巴彦淖尔","巴中","鸡西","乌海","临沧","海东","张掖","商洛","黑河","哈密","吴忠","攀枝花","双鸭山","阿克苏","石嘴山","阿拉善盟","海西","平凉","林芝","固原","武威","儋州","吐鲁番","甘孜","辽源","临夏","铜川","金昌","鹤岗","伊春","中卫","怒江","和田","迪庆","甘南","阿坝","大兴安岭","七台河","山南","日喀则","塔城","博尔塔拉","昌都","阿勒泰","玉树","海南","黄南","果洛","克孜勒苏","阿里","海北","那曲","三沙'

In [8]:

# for jack in a:
#     os.remove("../css破解/jack")
a

['873b6f82.woff', 'b6ce0b8f.woff', 'd1d35beb.woff']

In [12]:
os.chdir("D://360downloads//美团比赛//css破解//") #设置需要查看的目录D:\360downloads\美团比赛\css破解
a = [x for x in os.listdir()]
for jack in a:
    os.remove("D://360downloads//美团比赛//css破解//"+jack)

In [40]:
list0[74]

'乌海'

In [50]:
list0[119]

'三沙'